# Using Keras for GPU-accelerated training of neural nets
## CIFAR10/100 using Keras inbuild imagedatagenerator class

## An example model

As an example let's use a conv net doing supervised image classification on the CIFAR10 dataset 


In [1]:
import keras
from keras.datasets import cifar10
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D
import os

Using TensorFlow backend.


# How to specify GPUs for training
- use CUDA_VISIBLE_DEVICES environment variable

In [2]:
# set to "-1" to disable GPU
os.environ["CUDA_VISIBLE_DEVICES"]="0"

In [3]:
# import tensorflow as tf
# import keras.backend as K
# config = tf.ConfigProto(device_count={"CPU": 8})
# K.tensorflow_backend.set_session(tf.Session(config=config))

In [4]:

num_classes = 10
epochs = 100
data_augmentation = True
num_predictions = 20
# save_dir = os.path.join(os.getcwd(), 'saved_models')
# model_name = 'keras_cifar10_trained_model.h5'

In [5]:
# The data, split between train and test sets:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

x_train shape: (50000, 32, 32, 3)
50000 train samples
10000 test samples


In [6]:
# Convert class vectors to binary class matrices.
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

In [7]:
model = Sequential()
model.add(Conv2D(32, (3, 3), padding='same',
                 input_shape=x_train.shape[1:]))
model.add(Activation('relu'))
model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(64, (3, 3), padding='same'))
model.add(Activation('relu'))
model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes))
model.add(Activation('softmax'))

In [8]:
# initiate RMSprop optimizer
#opt = keras.optimizers.rmsprop(lr=0.0001, decay=1e-6)
opt = keras.optimizers.rmsprop(lr=0.0001, decay=1e-6)

# Let's train the model using RMSprop
model.compile(loss='categorical_crossentropy',
              optimizer=opt,
              metrics=['accuracy'])

In [9]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 32, 32, 32)        896       
_________________________________________________________________
activation_1 (Activation)    (None, 32, 32, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 30, 30, 32)        9248      
_________________________________________________________________
activation_2 (Activation)    (None, 30, 30, 32)        0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 15, 15, 32)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 15, 15, 32)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 15, 15, 64)        18496     
__________

In [10]:
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255

In [11]:
if not data_augmentation:
    print('Not using data augmentation.')
    model.fit(x_train, y_train,
              batch_size=batch_size,
              epochs=epochs,
              validation_data=(x_test, y_test),
              shuffle=True)
else:
    print('Using real-time data augmentation.')
    # This will do preprocessing and realtime data augmentation:
    datagen = ImageDataGenerator(
        featurewise_center=False, #False,  # set input mean to 0 over the dataset
        samplewise_center=False, #False,  # set each sample mean to 0
        featurewise_std_normalization=False, #False,  # divide inputs by std of the dataset
        samplewise_std_normalization=False, #False,  # divide each input by its std
        zca_whitening=False,  # apply ZCA whitening
        zca_epsilon=1e-06,  # epsilon for ZCA whitening
        rotation_range=0,  # randomly rotate images in the range (degrees, 0 to 180)
        # randomly shift images horizontally (fraction of total width)
        width_shift_range=0.1,
        # randomly shift images vertically (fraction of total height)
        height_shift_range=0.1,
        shear_range=0.,  # set range for random shear
        zoom_range=0.,  # set range for random zoom
        channel_shift_range=0.,  # set range for random channel shifts
        # set mode for filling points outside the input boundaries
        fill_mode='nearest',
        cval=0.,  # value used for fill_mode = "constant"
        horizontal_flip=True,  # randomly flip images
        vertical_flip=False,  # randomly flip images
        # set rescaling factor (applied before any other transformation)
        rescale=None,
        # set function that will be applied on each input
        preprocessing_function=None,
        # image data format, either "channels_first" or "channels_last"
        data_format=None,
        # fraction of images reserved for validation (strictly between 0 and 1)
        validation_split=0.0)

    # Compute quantities required for feature-wise normalization
    # (std, mean, and principal components if ZCA whitening is applied).
    datagen.fit(x_train)

Using real-time data augmentation.


## Training the model (with GPU)

Multiprocessing here refers to using multiple CPU threads to do the image preprocessing

Check out the effect that changing the parameters for the image data generator pipeline has on GPU load.

In particular, setting use_multiprocessing to False makes for a dramatic slowdown

*health warning* You should take the warning about Keras ImageDataGenerators not being thread safe seriously

In [14]:
batch_size = 1024

In [ ]:
# Fit the model on the batches generated by datagen.flow().
model.fit_generator(datagen.flow(x_train, y_train,
                                batch_size=batch_size),
                    steps_per_epoch=int(x_train.shape[0]/batch_size),
                    epochs=epochs,
                    validation_data=(x_test, y_test),
                    max_queue_size=16,
                    workers=8,
                    use_multiprocessing=False)

Epoch 1/100
48/48 [==============================] - 42s 873ms/step - loss: 1.2713 - acc: 0.5465 - val_loss: 1.1937 - val_acc: 0.5740
Epoch 2/100
48/48 [==============================] - 41s 860ms/step - loss: 1.2639 - acc: 0.5503 - val_loss: 1.1744 - val_acc: 0.5868
Epoch 3/100
48/48 [==============================] - 41s 862ms/step - loss: 1.2657 - acc: 0.5488 - val_loss: 1.1461 - val_acc: 0.5976
Epoch 4/100
48/48 [==============================] - 41s 847ms/step - loss: 1.2487 - acc: 0.5582 - val_loss: 1.1330 - val_acc: 0.5993
Epoch 5/100
48/48 [==============================] - 41s 847ms/step - loss: 1.2461 - acc: 0.5578 - val_loss: 1.1963 - val_acc: 0.5780
Epoch 6/100
48/48 [==============================] - 41s 862ms/step - loss: 1.2321 - acc: 0.5617 - val_loss: 1.1941 - val_acc: 0.5742
Epoch 7/100
48/48 [==============================] - 41s 861ms/step - loss: 1.2379 - acc: 0.5596 - val_loss: 1.1170 - val_acc: 0.6077
Epoch 8/100
48/48 [==============================] - 41s 861ms